In [1]:
import tensorflow as tf
import numpy as np

class NeuralNetworkLayer(object):
    '''
    A Neural Network Layer
    '''
    def __init__(self, input_size, output_size, name=None, tf_session=None):
        self.number_of_neurons = output_size
        self.inputs_per_neuron = input_size
        self.name = name
        prefix = self.name+'.' if self.name != None else 'layer_'
        self.synaptic_weights = tf.Variable(tf.random_normal([input_size, output_size]), name=prefix+'synaptic_weights')
        self.biases = tf.Variable(tf.random_normal([output_size]), name=prefix+'biases')
        self.tf_session = tf_session
    
    def set_tf_session(self, tf_session):
        self.tf_session = tf_session
    
    def __repr__(self):
        if(self.tf_session == None):
            return str({
                'number_of_neurons': self.number_of_neurons, 'inputs_per_neuron': self.inputs_per_neuron, 'name': self.name
            })
        else:
            return str({
                'number_of_neurons': self.number_of_neurons, 'inputs_per_neuron': self.inputs_per_neuron, 'name': self.name,
                'synaptic_weights': self.synaptic_weights.eval(session=self.tf_session),
                'biases': self.biases.eval(session=self.tf_session)
            })

class MultiLayerNeuralNetwork(object):
    '''
    A MultiLayer Neural Network
    '''
    def __init__(self, layer_sizes, input_type, output_type):
        # Placeholders
        self.x_ = tf.placeholder(input_type, [None, layer_sizes[0]], name='x_')
        self.y_ = tf.placeholder(output_type, [None, layer_sizes[-1]], name='y_')
        # Neural Network Layers
        self.layers = [ NeuralNetworkLayer(input_size, output_size, name='layer_{}'.format(i)) 
                       for (i, (input_size, output_size)) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])) ]
        # Prediction, loss and optimizer
        self.prediction = self.predict(self.x_)[-1]
        #self.loss = tf.reduce_mean(tf.squared_difference(self.y_, self.prediction))
        #self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(self.loss)
    
    def set_training_params(self, loss_function, optimizer):
        self.loss = loss_function(self.y_, self.prediction)
        self.optimizer = optimizer.minimize(self.loss)
    
    def predict(self, inputs):
        '''
        Returns the list of the outputs from each layer (from first to last)
        '''
        layer_outputs_stack = []
        layer_inputs = inputs # layer_inputs for next iteration
        for layer in self.layers:
            layer_outputs = tf.add(tf.matmul(layer_inputs, layer.synaptic_weights), layer.biases)
            layer_outputs = tf.nn.sigmoid(layer_outputs)
            layer_outputs_stack.append(layer_outputs)
            layer_inputs = layer_outputs
        return layer_outputs_stack
    
    def set_tf_session(self, tf_session):
        for layer in self.layers:
            layer.set_tf_session(tf_session)
    
    def __repr__(self):
        return str(self.layers)

In [2]:
tf.reset_default_graph()

# Creating the model
model = MultiLayerNeuralNetwork([3, 4, 3, 1], tf.float32, tf.float32)
print('Model before initializing weights')
print(model)

# The training set. We have 7 examples, each consisting of 3 input values and 1 output value
training_set_inputs = np.array([[0, 0, 1], [0, 1, 1], [1, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 1], [0, 0, 0]]).astype(np.float32)
training_set_outputs = np.array([[0, 1, 1, 1, 1, 0, 0]]).T.astype(np.float32)

# We will use MSE as loss function, and optimize with AdamOptimizer
loss_function = lambda y, pred: tf.reduce_mean(tf.squared_difference(y, pred))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
model.set_training_params(loss_function, optimizer)

# Launch the graph
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    # Initializing the variables
    sess.run(init)
    model.set_tf_session(sess)
    print(model)
    # Now training
    print("Training...")
    for epoch in range(100000):
        # Run optimization op (backprop) and loss op (to get loss value)
        (_, loss) = sess.run(
            [model.optimizer, model.loss], 
            feed_dict={model.x_: training_set_inputs, model.y_: training_set_outputs})
        # Display logs per epoch step
        if epoch % 1000 == 0:
            print('Epoch: {}, loss={}'.format(epoch, loss))
    print("Finished training!")
    print(model)
    
    # Just checking how it adjusts to the training set
    print("Predicting on the training set:")
    layer_outputs = model.predict(training_set_inputs)
    print(layer_outputs[-1].eval(session=sess))
    
    # Just checking how it adjusts to the training set
    print("Predicting on the training set:")
    test_set_inputs = np.array([[1, 1, 0]]).astype(np.float32)
    layer_outputs = model.predict(test_set_inputs)
    print(layer_outputs[-1].eval(session=sess))
    

Model before initializing weights
[{'number_of_neurons': 4, 'inputs_per_neuron': 3, 'name': 'layer_0'}, {'number_of_neurons': 3, 'inputs_per_neuron': 4, 'name': 'layer_1'}, {'number_of_neurons': 1, 'inputs_per_neuron': 3, 'name': 'layer_2'}]
[{'number_of_neurons': 4, 'inputs_per_neuron': 3, 'biases': array([ 0.86882907,  1.82817304, -0.79447335,  0.62770885], dtype=float32), 'name': 'layer_0', 'synaptic_weights': array([[-0.75024509, -0.30697507, -0.37605351, -1.88932753],
       [ 0.34027651,  1.36733377,  0.36677638,  0.28121865],
       [-0.7164005 , -0.05920342, -0.29389295,  0.40518087]], dtype=float32)}, {'number_of_neurons': 3, 'inputs_per_neuron': 4, 'biases': array([-1.16655111,  0.31444797, -1.38888073], dtype=float32), 'name': 'layer_1', 'synaptic_weights': array([[ 0.04697009, -1.58462417, -2.65292788],
       [ 1.14246023,  1.06249595,  1.42925096],
       [-0.24149585, -0.75541091, -1.01969469],
       [-0.55983084, -0.29572338,  0.07365815]], dtype=float32)}, {'number_of

In [3]:
# Having a look at the variables generated by tensorflow
for variable in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES) + tf.get_collection(tf.GraphKeys.SAVEABLE_OBJECTS):
    print(variable.name)

layer_0.synaptic_weights:0
layer_0.biases:0
layer_1.synaptic_weights:0
layer_1.biases:0
layer_2.synaptic_weights:0
layer_2.biases:0
beta1_power:0
beta2_power:0
layer_0.synaptic_weights/Adam:0
layer_0.synaptic_weights/Adam_1:0
layer_0.biases/Adam:0
layer_0.biases/Adam_1:0
layer_1.synaptic_weights/Adam:0
layer_1.synaptic_weights/Adam_1:0
layer_1.biases/Adam:0
layer_1.biases/Adam_1:0
layer_2.synaptic_weights/Adam:0
layer_2.synaptic_weights/Adam_1:0
layer_2.biases/Adam:0
layer_2.biases/Adam_1:0
